# Embedding word2vec fasttext

на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
Проверить насколько хорошо работают подходы

In [39]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import pandas as pd
import numpy as np
import re
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText

In [45]:
from corus import load_ods_gazeta
path = '/gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)

NewsRecord(
    timestamp=datetime.datetime(2008, 11, 21, 15, 19, 14),
    url='https://www.gazeta.ru/news/business/2008/11/21/n_1298950.shtml',
    edition=None,
    topics='Бизнес',
    authors=None,
    title='Госдума сокращает срок действия ставки экспортных пошлин на нефть',
    text='Госдума приняла сегодня в первом чтении и сразу в целом поправки в закон «О таможенном тарифе», сокращающие срок действия ставки экспортных пошлин на нефть с 2-х до 1-го месяца.nnДля установления средних цен на нефть марки Urals и расчета экспортных пошлин правительство России в течение двух месяцев проводит мониторинг на международных рынках нефтяного сырья (средиземноморском и роттердамском), экспортные пошлины на нефть устанавливаются также раз в два месяца.nnСокращение на месяц периода мониторинга (с 15-го числа каждого календарного месяца по 14-е число следующего месяца) и соответственно срока действия ставок экспортных пошлин «позволит более оперативно реагировать на изменения экономической сит

In [46]:
df = pd.DataFrame(records)

In [47]:
df.head()

,0,1,2,3,4,5,6,7
0,2012-09-27 10:54:51,https://www.gazeta.ru/science/news/2012/09/27/...,None,Наука,None,Японские физики повторили синтез 113-го элемента,Японские ученые из физического центра RIKEN за...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
1,2015-12-01 15:28:45,https://www.gazeta.ru/army/news/7957079.shtml,None,Армия,None,Times: Россия строит новую авиабазу в Сирии,Россия готовится расширить свою военную операц...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
2,2013-04-18 16:42:06,https://www.gazeta.ru/travel/news/2013/04/18/n...,None,Новости,None,Власти Египта гарантируют безопасность российс...,Египет ждет российских туристов и гарантирует ...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
3,2012-09-26 13:54:13,https://www.gazeta.ru/politics/news/2012/09/26...,None,Политика,None,Гордума Новочеркасска приняла отставку мэра го...,В Ростовской области депутаты гордумы Новочерк...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
4,2012-09-26 21:11:33,https://www.gazeta.ru/culture/news/2012/09/26/...,None,Культура,None,"Министерство культуры назвало фильмы, на котор...",Министерство культуры России определило перече...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."


In [48]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [49]:
assert True

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

for line in df[6]:
    spls = preprocess_txt(line)
    sentences.append(spls)
    c += 1
    if c > 100000:
        break

In [50]:
sentences = [i for i in sentences if len(i) > 2]

In [51]:
sentences[0]

['японский',
 'учёный',
 'физический',
 'центр',
 'riken',
 'заявлять',
 'удаться',
 'синтезировать',
 'атом',
 '113го',
 'элемент',
 'таблица',
 'менделеев',
 'элемент',
 'впервые',
 'получить',
 '2003',
 'российский',
 'американский',
 'физик',
 'эксперимент',
 'синтез',
 '115го',
 'элемент',
 'открытие',
 'признать',
 'международный',
 'союз',
 'теоретический',
 'прикладной',
 'химия',
 'сообщаетnриа',
 '«новости»nnть',
 'природа',
 'существовать',
 'элемент',
 'атомный',
 'номер',
 'число',
 'протон',
 'ядро',
 'атом',
 'большой',
 '92',
 'тяжёлый',
 'уран',
 'тяжёлый',
 'элемент',
 'например',
 'плутоний',
 'нарабатываться',
 'атомный',
 'реактор',
 'элемент',
 'тяжёлый',
 '100го',
 'фермий',
 'получать',
 'ускоритель',
 'путём',
 'бомбардировка',
 'мишень',
 'тяжёлый',
 'ион',
 'слияние',
 'ядро',
 'мишень',
 '«снаряда»',
 'возникать',
 'ядро',
 'новый',
 'элементаnnгруппа',
 'учёный',
 'руководство',
 'косука',
 'морить',
 'статья',
 'опубликовать',
 'journal',
 'of',
 'physical

In [52]:
modelW2V = Word2Vec(sentences=sentences, size=300, window=5, min_count=3)

In [53]:
modelFT = FastText(sentences=sentences, size=300, min_count=3, window=5, workers=8)

In [55]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

counter = 0
index_map = {}

for line in df[6]:
    n_w2v = 0
    n_ft = 0
    index_map[counter] = line
    text = preprocess_txt(line)
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in text:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

True

In [56]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [57]:
TEXT = "какой город самы красивый"

In [58]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['В эти выходные Москва отмечает свое 870-летие. Юбилей Москвы празднуют всего в четвертый раз за всю историю города. Впервые это произошло еще 170 лет назад, в те времена, когда столицей России был Санкт-Петербург. «Газета.Ru» рассказывает, как чествовали Москву в разные века и как складывались традиции праздника.nВпервые юбилей Москвы отмечали в 1847 году. Идею отпраздновать 700-летие города продвигали интеллектуалы-славянофилы, которые были заодно и «москвофилами». В те времена еще спорили о том, в каком именно году основали Москву, и, тем более, не было конкретного дня, на который можно было бы назначить торжества. Историки предлагали разные даты и в итоге остановились на весне 1847 года.nПрограмма подготовки к юбилею включала мероприятия, которые вполне могли бы сопровождать День города и в наши дни: художникам предложили написать картины с видами Москвы, историкам — создать описания ее церквей, монастырей, Московского университета, почтамта и других учреждений, а также — быта, нр

In [59]:
get_response(TEXT, ft_index, modelFT, index_map)

['В эти выходные Москва отмечает свое 870-летие. Юбилей Москвы празднуют всего в четвертый раз за всю историю города. Впервые это произошло еще 170 лет назад, в те времена, когда столицей России был Санкт-Петербург. «Газета.Ru» рассказывает, как чествовали Москву в разные века и как складывались традиции праздника.nВпервые юбилей Москвы отмечали в 1847 году. Идею отпраздновать 700-летие города продвигали интеллектуалы-славянофилы, которые были заодно и «москвофилами». В те времена еще спорили о том, в каком именно году основали Москву, и, тем более, не было конкретного дня, на который можно было бы назначить торжества. Историки предлагали разные даты и в итоге остановились на весне 1847 года.nПрограмма подготовки к юбилею включала мероприятия, которые вполне могли бы сопровождать День города и в наши дни: художникам предложили написать картины с видами Москвы, историкам — создать описания ее церквей, монастырей, Московского университета, почтамта и других учреждений, а также — быта, нр